# Normalización



In [1]:
import nltk
import re
import string
from pprint import pprint
corpus = ["The brown fox wasn't that quick and he couldn't win the race",
          "Hey that's a great deal! I just bought a phone for $199",
          "@@You'll (learn) a **lot** in the book. Python is an amazing language!@@"]

## Limpieza del texto

Los datos a utilizar pueden contener caracteres o tokens innecesarios, por lo que deben ser eliminados antes de realizar alguna operación como la tokenización o normalización. Por ejemplo, etiquetas HTML, XML o JSON. 

Existen distintas formas de limpiar el texto con funciones como `clean_html()` de `nltk` o con la librería `BeautifulSoup`. También se pueden utilizar expresiones regulares, xpath o la librería lxml.



In [2]:
def tokenize_text(text):
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentence) for sentence in sentences]
    return word_tokens

In [3]:
token_list = [tokenize_text(text) for text in corpus] 
pprint(token_list)

[[['The',
   'brown',
   'fox',
   'was',
   "n't",
   'that',
   'quick',
   'and',
   'he',
   'could',
   "n't",
   'win',
   'the',
   'race']],
 [['Hey', 'that', "'s", 'a', 'great', 'deal', '!'],
  ['I', 'just', 'bought', 'a', 'phone', 'for', '$', '199']],
 [['@',
   '@',
   'You',
   "'ll",
   '(',
   'learn',
   ')',
   'a',
   '*',
   '*',
   'lot',
   '*',
   '*',
   'in',
   'the',
   'book',
   '.'],
  ['Python', 'is', 'an', 'amazing', 'language', '!'],
  ['@', '@']]]


### Eliminación caracteres especiales

Una tarea previa a la normalización del texto es eliminar los caracteres especiales. Por ejemplo símbolos especiales, signos de puntuación. Este paso regularmente es ejecutado antes o después de la tokenización. La razón de este paso es eliminar elementos que no tienen significado cuando se analiza el texto, o se extraen características o información basada en NLP o ML. 

El siguiente código muestra la eleminición de caracteres especiales después de la tokenización:

In [4]:
def remove_characters_after_tokenization(tokens):
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation))) 
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens]) 
    return filtered_tokens

In [5]:
filtered_list_1 =  [filter(None,[remove_characters_after_tokenization(tokens) for tokens in sentence_tokens]) for sentence_tokens in token_list]
pprint(list(filtered_list_1))

El atributo `string.puntuation` consiste en todos los símbolos y caracteres especiales, y crear patrones de expresiones regulares de estos. La función `filter` ayuda a remover los tokens vacios obtenidos después de revomer token de caracteres especiales utilizando el método `reg sub`.


In [6]:
def remove_characters_before_tokenization(sentence, keep_apostrophes=False):
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]' # add other characters here to remove them
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    else:
        PATTERN = r'[^a-zA-Z0-9 ]' # only extract alpha-numeric characters
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    return filtered_sentence

In [7]:
filtered_list_2 = [remove_characters_before_tokenization(sentence) for sentence in corpus]
print(filtered_list_2)

['The brown fox wasnt that quick and he couldnt win the race', 'Hey thats a great deal I just bought a phone for 199', 'Youll learn a lot in the book Python is an amazing language']


In [8]:
cleaned_corpus = [remove_characters_before_tokenization(sentence, keep_apostrophes=True) for sentence in corpus]
print(cleaned_corpus)

["The brown fox wasn't that quick and he couldn't win the race", "Hey that's a great deal! I just bought a phone for 199", "You'll learn a lot in the book. Python is an amazing language!"]


## Expasión de contracciones

Las contracciones son versiónes cortas de palabras o silabas. En inglés los constracciones son creadas removiendo vocales de la palabra. Por ejemplo, is not - isn't, will not - won't. Usualmente las constracciones son evitadas en escritura formal, pero en la escritura informal son muy comunes. Existen varias formas de contracción que esta ligadas al tipo de verbo auxiliar que da una forma de contracción normal, negada u otras contracciones coloquiales especiales, algunas de ellas no implican auxiliares. 

Las contracciones implican un problema en NLP y análisis de texto, ya que contiene el caracter especial apostrofe en la palabra. Además, se tiene dos o más palabras representadas en la contracción, espo amplia las formas que se puede tokenizar una palabra. Idealmente, se puede proponer un mapeo de contracciones y sus correspondientes expansiones.

Se puede crear un archivo `contractions.py` en un diccionario de Python.

````Python
CONTRACTION_MAP = {
"isn't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
.
.
.
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}
````
Se debe considera que algunas de las contracciones tienen multiplés formas. Por ejemplo, *you'll* puede indicar *you will* o *you shall*.

In [17]:
from contractions import CONTRACTION_MAP
def expand_contractions(sentence, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
    expanded_sentence = contractions_pattern.sub(expand_match, sentence)
    return expanded_sentence

In [18]:
expanded_corpus = [expand_contractions(sentence, CONTRACTION_MAP) for sentence in cleaned_corpus]

print(expanded_corpus)

["The brown fox wasn't that quick and he couldn't win the race", "Hey that's a great deal! I just bought a phone for 199", "You'll learn a lot in the book. Python is an amazing language!"]


## Mayúsculas y minúsculas

In [11]:
print(corpus[0].lower())

the brown fox wasn't that quick and he couldn't win the race


In [12]:
print(corpus[0].upper())

THE BROWN FOX WASN'T THAT QUICK AND HE COULDN'T WIN THE RACE


## Palabras vacías


In [14]:

def remove_stopwords(tokens):
    stopword_list = nltk.corpus.stopwords.words('english')
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    return filtered_tokens

In [15]:
expanded_corpus_tokens = [tokenize_text(text) for text in expanded_corpus]

filtered_list_3 =  [[remove_stopwords(tokens) for tokens in sentence_tokens] for sentence_tokens in expanded_corpus_tokens]
print(filtered_list_3)

NameError: name 'expanded_corpus' is not defined